In [ ]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division
%matplotlib inline 

# outline 
1. [Get tree data](#treedata)
1. [Combine tree data and genotype data](#combine)

<a id="treedata"></a>
# get tree location data from master data, add X Y to geno df

In [ ]:
#get tree location data from master data 
filE = '/home/lindb/teakettle/data/trees_with_converted.txt'
t = pd.read_csv(filE,sep='\t')
t.head()

In [ ]:
#keep only necessary columns
grid = ['converted','seedling','x','y','z','xutm','yutm','zutm','dbh_11','dbh_04','dbh_13']
info = t.loc[:,grid]
info.head()

In [ ]:
#make sure no duplicates
nrow(info) == luni(info['converted'])

In [ ]:
#make sure either True or MD (missing data)
uni(t['sampled'])

In [ ]:
'bc1_new010' in info['converted'].tolist()

In [ ]:
for row in info.index:
    if info.loc[row,'seedling'] == False:
        if math.isnan(info.loc[row,'dbh_11']):
            if math.isnan(info.loc[row,'dbh_04']) == False:
                print '%s given regression' % info.loc[row,'converted']
                info.loc[row,'dbh_11'] = info.loc[row,'dbh_04']*0.9699+5.1995 #lm(dbh_11 ~ dbh_04)
            else:
                print "%s given median" % info['converted'][row] 
                info.loc[row,'dbh_11'] = 28.37125 #(median + mean)/2 -- 6 trees

<a id="combine"></a>
# combine dfs

In [ ]:
# get genotype calls
g = pd.read_csv('/home/lindb/teakettle/genemarker/results_after_jan.tsv',sep='\t')
g.index = g['samp']
g.head()

In [ ]:
info.index = info['converted']
info.head()

In [ ]:
c = pd.merge(g,info,left_index=True,right_index=True)
c.head()

In [ ]:
len(c.columns)

In [ ]:
ccols = ['samp',
 'x',
 'y',
 'dbh_11',
 'xutm',
 'yutm',
 'seedling',
 'rps12',
 'rps12_2',
 'rps2',
 'rps2_2',
 'rps39',
 'rps39_2',
 'rps50',
 'rps50_2',
 'pc10',
 'pt71936',
 'pt87268']
c = c[ccols]
c.index = range(nrow(c))
c.head()

In [ ]:
nrow(c) == luni(c['samp'])*2

In [ ]:
c.head()

# double check each tree has all necessary data

In [ ]:
# these samples weren't in the tfk data set. Likely just outside of a plot
# therefore these don't have either 'seedling' or x/y/xutm/yutm coordinates
badtrees = ['uc2_3141','uc2_5114','uc2_5185','uc2_9554','uc2_9592','un1_3964','un1_5774','un3_3258']
newc = c[~c['samp'].isin(badtrees)]
nrow(newc)

In [ ]:
nrow(c) - 2*len(badtrees) == nrow(newc)

In [ ]:
len(badtrees)

# if 'new' in samp, assert seedling == True

In [ ]:
uni(newc['seedling']) # {True,False}

In [ ]:
np.where(newc.columns == 'seedling')[0][0]

In [ ]:
newc.columns[6] # 'seedling'

In [ ]:
for row in newc.index:
    if newc.loc[row,'seedling'] == False:
        if 'new' in newc.loc[row,'samp']:
#             print newc.loc[row,'samp']
            newc.loc[row,newc.columns[6]] = True

In [ ]:
nrow(newc) == 2*luni(newc['samp'])

# write file

In [ ]:
# # i updated the data
# src = '/home/lindb/teakettle/data/'
# dst = '/home/lindb/teakettle/data_old'
# if not op.exists(dst):
#     mv(src,dst)

DIR = '/home/lindb/teakettle/data/moran'
if not op.exists(DIR):
    os.makedirs(DIR)
newc.to_csv(op.join(DIR,'moran_infile_final_jan.txt'),header=True,index=False,sep='\t')